In [1]:
from question_types.sparql import SparqlQueries
from rdflib.namespace import RDFS
import pandas as pd
import unicodedata
import re

In [2]:
sparql = SparqlQueries("../dataset/14_graph.nt")

2024-11-28 14:40:29,597 | INFO | __init__ | Initializing SparqlQueries class...
2024-11-28 14:41:14,704 | INFO | __init__ | Graph parsed
2024-11-28 14:41:14,704 | INFO | __init__ | ...SparqlQueries class initialized successfully.


In [3]:
def normalize_string(s):
    """Cleans the input entity to a uniform naming convention, by removing non ascii characters, encoding it to utf, setting it to lowercase, and removing redundant spaces"""
    s = s.lower()
    s = unicodedata.normalize('NFKD', s)
    s = s.encode('ascii', 'ignore').decode('utf-8')
    s = re.sub(r'[^\w\s]', '', s)
    s = ' '.join(s.split())
    return s

In [4]:
def get_label(graph, uri):
    """Function to get the rdfs:label or schema:name for a URI if available."""
    for label in graph.objects(uri, RDFS.label):
        return normalize_string(str(label))
    
    schema_name = "http://schema.org/name"
    
    for label in graph.objects(uri, graph.namespace_manager.compute_qname(schema_name)[2]):
        return normalize_string(str(label))
    
    return decode_uri(uri)

schema_mapping = {
    "http://ddis.ch/atai/": "ddis",
    "http://www.wikidata.org/entity/": "wd",
    "http://www.wikidata.org/prop/direct/": "wdt",
    "http://schema.org/": "schema"
}

def decode_uri(uri):
    """Helper function to replace full URIs with prefixes and extract local names"""
    for schema, prefix in schema_mapping.items():
        if uri.startswith(schema):
            return f"{prefix}:{uri[len(schema):]}" 
    return uri

triples_data = [(subject_id, predicate_id, object_id) for subject_id, predicate_id, object_id in sparql.graph]

df_triples = pd.DataFrame(triples_data, columns=['subject_id', 'predicate_id', 'object_id'])

In [5]:
subject_as_object_set = set(df_triples['subject_id']).intersection(set(df_triples['object_id']))
print("Stage 1 - subject_ids that are also objects:", len(subject_as_object_set))

object_to_subjects_predicates = {}
for index, row in df_triples.iterrows():
    if row['object_id'] in subject_as_object_set:
        if row['object_id'] not in object_to_subjects_predicates:
            object_to_subjects_predicates[row['object_id']] = []
        object_to_subjects_predicates[row['object_id']].append((row['subject_id'], row['predicate_id']))

new_triples_data = df_triples.values.tolist()
for index, row in df_triples.iterrows():
    if row['subject_id'] in object_to_subjects_predicates:
        new_triples_data.extend([(row['subject_id'], predicate, new_object) for new_object, predicate in object_to_subjects_predicates[row['subject_id']]])

print("Stage 2 - Newly added triples:", len(new_triples_data) - len(df_triples))

df_new_triples = pd.DataFrame(new_triples_data, columns=['subject_id', 'predicate_id', 'object_id'])
df_new_triples.head()

Stage 1 - subject_ids that are also objects: 136263
Stage 2 - Newly added triples: 48741780


,subject_id,predicate_id,object_id
0,http://www.wikidata.org/entity/Q64900128,http://www.wikidata.org/prop/direct/P495,http://www.wikidata.org/entity/Q881
1,http://www.wikidata.org/entity/Q740143,http://www.wikidata.org/prop/direct/P1981,http://www.wikidata.org/entity/Q20644796
2,http://www.wikidata.org/entity/Q312885,http://www.w3.org/2000/01/rdf-schema#label,James Marsters
3,http://www.wikidata.org/entity/Q1090696,http://www.wikidata.org/prop/direct/P18,https://commons.wikimedia.org/wiki/File:Over-d...
4,http://www.wikidata.org/entity/Q1895135,http://www.wikidata.org/prop/direct/P27,http://www.wikidata.org/entity/Q40


In [6]:
# NEW CELL, IMPORT IN NEXT VERSION
df_new_triples = df_new_triples.drop_duplicates()
df_new_triples.to_pickle("./exports/df_new_triples_only_ids.pkl")


In [7]:
df_new_triples = df_new_triples.drop_duplicates()

df_new_triples['subject_label'] = df_new_triples['subject_id'].apply(lambda x: get_label(sparql.graph, x))
df_new_triples['predicate_label'] = df_new_triples['predicate_id'].apply(lambda x: get_label(sparql.graph, x))
df_new_triples['object_label'] = df_new_triples['object_id'].apply(lambda x: get_label(sparql.graph, x))

df_grouped_new = df_new_triples.groupby(['subject_id', 'subject_label', 'predicate_label'])['object_label'].agg(lambda x: ','.join(x)).reset_index()

df_grouped_new = df_grouped_new.drop_duplicates()

df_grouped_new = df_grouped_new[df_grouped_new['subject_label'] != ""]
df_grouped_new = df_grouped_new[df_grouped_new['subject_label'].str.len() > 3]

# print(df_grouped_new[df_grouped_new['subject_id'].str.contains('Q13909')])

In [8]:
df_grouped_new.to_pickle("./exports/extended_graph_triples.pkl")
df_grouped_new.to_csv("./exports/extended_graph_triples.csv")